In [1]:
import os
import time
import numpy as np
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta
import plotly.offline as py
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Layout
py.init_notebook_mode(connected=True)

In [2]:
import getpass
token = getpass.getpass('token:')
pro = ts.pro_api(token)

token: ························································


# 获取当天日期
```python
Parameters: None

Returns: str ('20190325')
```

In [3]:
get_date = lambda :time.strftime('%Y%m%d',time.localtime(time.time()))

# 获取前N天日期
```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    N: int 前 N 天，默认 1 天

Returns: str ('20190324')
```

In [4]:
get_pastdate = lambda date=get_date(), N=1 :datetime.strftime(pd.to_datetime(date) - timedelta(N+1-1), '%Y%m%d')

# 获取最近N天所有股票的数据
```python
Parameters:

    date: str ('20190325') 默认今天
        
    N: int 最近 N 天，默认 1 天
        
    print_mode: bool 是否要打印，默认 False

Returns: list 每一个项都是那一天的 A 股所有股票的 OHLC 等数据构成的 DataFrame，DataFrame.name 为是日日期，首项[0]的日期最早，末项[-1]的日期最晚
```

In [5]:
def get_allNdays(date=get_date(), N=1, print_mode=False):
    data = []
    for _ in range(N):
        today = pro.daily(trade_date=date)
        while len(today)==0:
            date = get_pastdate(date=date, N=1)
            today = pro.daily(trade_date=date)
        if print_mode: print('【%s】'%date,end='\t')
        today['state'] = today['close']>=today['open'] # 阳柱:True, 阴柱:False
        today.name = date
        today.index = today['ts_code']
        data.append(today)
        date = get_pastdate(date=date, N=1)
    data.reverse()
    return data

# 获取最近N天单只个股的数据
```python
Parameters:
    
    code: str ('000001.SZ') 没有默认
        
    date: str ('20190325') 默认今天
        
    peroid: int 从 date 往前几天，默认 300 天
        
    print_mode: bool 是否要打印，默认 False

Returns: DataFrame 该支股票的历史数据
```

In [6]:
def get_oneNdays(code, date=get_date(), period=300):
    start_date = get_pastdate(date=date, N=period)
    name = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')
    df = pro.daily(ts_code=code, start_date=start_date, end_date=date).iloc[::-1]
    df['state'] = df['close']>=df['open'] # 阳柱:True, 阴柱:False
    df.index = range(len(df))
    df.name = '%s-%s'%(name[name['ts_code']==code]['name'].item(),code)
    return df

# 筛选倍量柱
```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    v_pct_min: float 今天的量比上昨天的量的最小值，默认 0.9
        
    v_pct_max: float 今天的量比上昨天的量的最大值，默认 1.1
        
    c_pct_min: float 今天的收盘价比上昨天的收盘价的最小值，默认 1.01
        
    ignore: list 名单中的每一项都是股票代码，只要在名单中就完全不考虑，默认 []
        
    print_mode: bool 是否要打印，默认 False

Returns: dict keys 为是日日期， values 为 list 每一个项都是满足条件的股票代码
```

In [7]:
def get_blz(date=get_date(), v_pct_min=0.9, v_pct_max=1.1, c_pct_min=1.01, ignore=[], print_mode=False):
    result = {date:[]}
    [yesterday,today] = get_allNdays(date=date, N=2)
    for code in today['ts_code']:
        if code in list(yesterday['ts_code']) and code not in ignore:
            today_v = float(today.loc[code]['vol'])
            today_c = float(today.loc[code]['close'])
            yesterday_v = float(yesterday.loc[code]['vol'])
            yesterday_c = float(yesterday.loc[code]['close'])
            v_pct = today_v/yesterday_v
            c_pct = today_c/yesterday_c
            cond1 = v_pct > v_pct_min and v_pct < v_pct_max
            cond2 = c_pct > c_pct_min
            if cond1 and cond2:
                if print_mode:print('{}\tv:{}\tc:{}'.format(code,round(v_pct,2),round(c_pct,2)))
                result[date].append(code)
    return result

# 筛选黄金柱

# 将军柱（价平量平）（量柱和价柱对比形成小平台）

1. 【阳胜柱】：所谓阳胜柱，就是量柱和价柱双向胜过其左侧最近的阴柱。

>a. 阳柱收盘高于阴柱开盘

>b. 阳柱的量高于阴柱的量

2. 【收盘价三日不破底】：不破实底就是基柱后三日的收盘价，没有跌破基柱的实底（开盘价）。这里有两种情况：

>a. 基柱后三日的收盘价，均高于或等于基柱实底（开盘价）

>b. 基柱后三日的收盘价平均值，不低于基柱实底（开盘价）

3. 【量柱群三日不过头】两种情况：

>a. 基柱后三日的量柱应逐步缩小（有时包括基柱）

>b. 基柱后三日的量柱可参差不齐，但量价之比相对持平。

# 黄金柱（价升量缩）（量柱和价柱对比形成喇叭口）

1. 【将军柱】

2. 【收盘价三日不破顶】：不破实顶就是基柱后三日的收盘价，没有跌破基柱实顶（收盘价）。这有两种情况：

>a. 基柱后三日的收盘价，均高于或等于基柱实顶（收盘价）

>b. 基柱后三日的收盘价平均值，不低于基柱实顶（收盘价）

3. 【量柱群三日不抬头】

>a. 基柱后三日的量柱应逐步缩小（有时包括基柱）

>b. 基柱后三日的量柱可参差不齐，但量价之比相对加大。

4. 【先者优先】

[5.2：伏击涨停的王牌之将军柱的三个标准](https://www.guminwangxiao.com/jiqiao/fjzt/1043.html)

[5.3：伏击涨停的王牌之黄金柱的三个标准](https://www.guminwangxiao.com/jiqiao/fjzt/1044.html)

```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    ignore: list 名单中的每一项都是股票代码，只要在名单中就完全不考虑，默认 []
        
    print_mode: bool 是否要打印，默认 False

Returns: dict keys 为是日日期， values 为 list 每一个项都是满足条件的股票代码
```

In [8]:
def get_hjz(date=get_date(), ignore=[], print_mode=False):
    result = {date:[]}
    [d1, d2, d3] = get_allNdays(date=date, N=3) # d1→d2→d3
    for code in d1['ts_code']:
        if code in list(d2['ts_code']) and code in list(d3['ts_code']) and code not in ignore:
            o1 = d1.loc[code]['open'].item()
            c1,c2,c3 = d1.loc[code]['close'].item(),d2.loc[code]['close'].item(),d3.loc[code]['close'].item()
            v1,v2,v3 = d1.loc[code]['vol'].item(),d2.loc[code]['vol'].item(),d3.loc[code]['vol'].item()
            cond_open = o1 < min(c2,c3)        # 收盘价三日不破底
            cond_close = c1 < min(c2,c3)       # 收盘价三日不破顶
            cond_vol = (v1 > v2) & (v2 > v3)   # 量柱群三日不过头、不抬头
            if cond_open and cond_close and cond_vol:
                if print_mode:print(code)
                result[date].append(code)
    return result

# 缩量群 / 平量群 / 梯量群
```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    peroid: int 缩量群的量柱数目，默认 6
        
    rough: 精确的模糊，即今天的价格要比昨天的 0.99 高，今天的量要比昨天的 1/0.99 低，默认 0.99 （以缩量群为例）
        
    ignore: list 名单中的每一项都是股票代码，只要在名单中就完全不考虑，默认 []
        
    print_mode: bool 是否要打印，默认 False

Returns: dict keys 为是日日期， values 为 list 每一个项都是满足条件的股票代码
```

In [9]:
def get_slq(date=get_date(), period=6, rough=0.99, ignore=[], print_mode=False):
    result = {date:[]}
    days = get_allNdays(date=date, N=period) # days[0]→days[1]→days[2]
    for code in days[0]['ts_code']:
        In = code not in ignore
        for day in days:
            if not In:
                break
            else:
                In *= code in list(day['ts_code'])
        if not In:
            break
        else:
            price_increase = True
            vol_decrease = True
            for i in range(len(days)-1):
                if price_increase:
                    price_increase *= days[i+1].loc[code]['close'].item()>days[i].loc[code]['close'].item()*rough
                else:
                    break
                if vol_decrease:
                    vol_decrease *= days[i+1].loc[code]['vol'].item()<days[i].loc[code]['vol'].item()/rough
                else:
                    break
            ##### N天至少价格要上涨N个点 #####
            price_increase *= days[-1].loc[code]['close'].item()>days[0].loc[code]['close'].item()*(1+period*0.01)
            ##### N天至少量要下降N个点 #####
            vol_decrease *= days[-1].loc[code]['vol'].item()<days[0].loc[code]['vol'].item()/(1+period*0.01)
            if price_increase and vol_decrease:
                if print_mode:print(code)
                result[date].append(code)
            else:
                pass
    return result

In [46]:
def get_plq(date=get_date(), period=6, rough=0.99, ignore=[], print_mode=False):
    result = {date:[]}
    days = get_allNdays(date=date, N=period) # days[0]→days[1]→days[2]
    for code in days[0]['ts_code']:
        In = code not in ignore
        for day in days:
            if not In:
                break
            else:
                In *= code in list(day['ts_code'])
        if not In:
            break
        else:
            price_increase = True
            vol_decrease = True
            for i in range(len(days)-1):
                if price_increase:
                    price_increase *= days[i+1].loc[code]['close'].item()>days[i].loc[code]['close'].item()*rough
                else:
                    break
                if vol_decrease:
                    vol_decrease *= days[i].loc[code]['vol'].item()*rough<days[i+1].loc[code]['vol'].item()<days[i].loc[code]['vol'].item()/rough
                else:
                    break
            ##### N天至少价格要上涨N个点 #####
            price_increase *= days[-1].loc[code]['close'].item()>days[0].loc[code]['close'].item()*(1+period*0.01)
            ##### N天的要在正负N个点之间 #####
            vol_decrease *= days[0].loc[code]['vol'].item()/(1+period*0.01)<days[-1].loc[code]['vol'].item()<days[0].loc[code]['vol'].item()*(1+period*0.01)
            if price_increase and vol_decrease:
                if print_mode:print(code)
                result[date].append(code)
            else:
                pass
    return result

In [53]:
def get_tlq(date=get_date(), period=6, rough=0.99, ignore=[], print_mode=False):
    result = {date:[]}
    days = get_allNdays(date=date, N=period) # days[0]→days[1]→days[2]
    for code in days[0]['ts_code']:
        In = code not in ignore
        for day in days:
            if not In:
                break
            else:
                In *= code in list(day['ts_code'])
        if not In:
            break
        else:
            price_increase = True
            vol_decrease = True
            for i in range(len(days)-1):
                if price_increase:
                    price_increase *= days[i+1].loc[code]['close'].item()>days[i].loc[code]['close'].item()*rough
                else:
                    break
                if vol_decrease:
                    vol_decrease *= days[i+1].loc[code]['vol'].item()>days[i].loc[code]['vol'].item()*rough
                else:
                    break
            ##### N天至少价格要上涨N个点 #####
            price_increase *= days[-1].loc[code]['close'].item()>days[0].loc[code]['close'].item()*(1+period*0.01)
            ##### N天至少量要下降N个点 #####
            vol_decrease *= days[-1].loc[code]['vol'].item()>days[0].loc[code]['vol'].item()*(1+period*0.01)
            ##### 价格上升角度大于量上升的角度
            p_vs_v = (days[-1].loc[code]['close'].item()-days[0].loc[code]['close'].item())>(days[-1].loc[code]['vol'].item()-days[0].loc[code]['vol'].item())
            if price_increase and vol_decrease and p_vs_v:
                if print_mode:print(code)
                result[date].append(code)
            else:
                pass
    return result

# N天中满足条件的次数
```python
Parameters:
    
    Func: 需要满足的条件（函数），例如：get_blz、get_hjz、get_slq，默认 get_blz
        
    N: int N天中，默认 10
        
    Parameters: dict 相应的 Func 所传入的参数，默认为 {'date':get_date()}
        
    ignore: list 名单中的每一项都是股票代码，只要在名单中就完全不考虑，默认 []
        
    print_mode: bool 是否要打印，默认 False

Returns: dict keys 是股票代码(已剔除没有任何天满足条件的股票)，values 是 N 天满足条件的次数
```

In [10]:
def countN(func=get_blz, N=10, parameters={'date':get_date()}, ignore=[]):
    all_stock = pro.stock_basic(exchange='', list_status='L', fields='ts_code')['ts_code']
    count = dict(zip(all_stock,[0]*len(all_stock)))
    for _ in range(N):
        satisfied = func(**parameters)
        for code in list(satisfied.values())[0]:
            count[code] += 1
        parameters['date'] = get_pastdate(list(satisfied.keys())[0])
    return {code: times for code, times in count.items() if times>0}

# 画K线图 OHLC
```python
Parameters:
    
    code: str ('000001.SZ') 没有默认
        
    date: str ('20190325') 默认今天
        
    peroid: int 从 date 往前几天，默认 300 天

Returns: DataFrame 该支股票的历史数据，同时在同级目录创建档名为股票代码的 html 绘图档
```

In [60]:
def plotly_ohlc(code, date=get_date(), period=300):
    
    df = get_oneNdays(code, date, period)
    
    INCREASING_COLOR = '#e2002a'
    DECREASING_COLOR = '#19975d'
    
    trace = go.Ohlc(x=pd.to_datetime(df['trade_date']),
                open=df.open,
                high=df.high,
                low=df.low,
                close=df.close,
                yaxis = 'y2',
                name = 'OHLC',
                increasing=dict(line=dict(color=INCREASING_COLOR)),
                decreasing=dict(line=dict(color=DECREASING_COLOR)),
                text = df['pct_chg']
    )
    data = [trace]

    layout = go.Layout(
        xaxis = dict(rangeslider = dict(visible = False)),
        title = df.name,
        titlefont = dict(size = 30),
        yaxis = dict( domain = [0, 0.2], showticklabels = True),
        yaxis2 = dict( domain = [0.2, 0.97] ),
        legend = dict( orientation = 'h', y=0.97, x=0, yanchor='bottom'),
        margin = dict( t=60, b=40, r=40, l=40 ),
        plot_bgcolor = 'rgb(250, 250, 250)'
    )
    fig = dict(data=data, layout=layout)
    
    colors = list(df['state'].map({True:INCREASING_COLOR,False:DECREASING_COLOR}))

    fig['data'].append( dict( x=pd.to_datetime(df['trade_date']), y=df.vol,                         
                             marker=dict( color=colors ),
                             type='bar', yaxis='y', name='vol' ) )

    name = '{}.html'.format(df.name.split('-')[1])
    py.plot(fig, filename = name)
    return df

# 画N天K线图 OHLC
```python
Parameters:
    
    code: str ('000001.SZ') 没有默认
        
    date: str ('20190325') 默认今天
        
    peroid: int 从 date 往前几天，默认 300 天
        
    N: int N 天数据画一根柱子，默认 2 天

Returns: list 该支股票的历史数据以 N 天划分，例如：[0] 表示01、23、34，[1] 表示12、34、56
```

In [12]:
def Nplotly_ohlc(code, date=get_date(), period=300, N=2):
    
    def NK(code, date, period, N):
        def nk(df,N,start):
            df3 = pd.DataFrame(columns=['trade_date','open','high','low','close','vol','amount','state'])
            for i in range(start,len(df)-N+1,N):
                df3.loc[i+N-1,'trade_date'] = df.loc[i,'trade_date']
                df3.loc[i+N-1,'open'] = df.loc[i,'open']
                df3.loc[i+N-1,'high'] = max(df.loc[i:i+N-1,'high'])
                df3.loc[i+N-1,'low'] = min(df.loc[i:i+N-1,'low'])
                df3.loc[i+N-1,'close'] = df.loc[i+N-1,'close']
                df3.loc[i+N-1,'vol'] = sum(df.loc[i:i+N-1,'vol'])
                df3.loc[i+N-1,'amount'] = sum(df.loc[i:i+N-1,'amount'])
                df3.loc[i+N-1,'state'] = df.loc[i+N-1,'close']>df.loc[i,'open']
            df3.name = df.name
            return df3
        df_list = []
        for i in range(N):
            df_list.append(nk(df=get_oneNdays(code=code,date=date,period=period), N=N, start=i))
        return df_list
    
    df_list = NK(code, date, period, N)
    
    INCREASING_COLOR = ['#e2002a']*3#,'#f8b4b4','#ea513b']*2
    DECREASING_COLOR = ['#19975d']*3#,'#b3e0af','#92bc4e']*2
    
    data = []
    colors = []
    
    for i in range(len(df_list)):
        idict = go.Ohlc(
        open = df_list[i].open,
        high = df_list[i].high,
        low = df_list[i].low,
        close = df_list[i].close,
        x = pd.to_datetime(df_list[i]['trade_date']),
        yaxis = 'y2',
        name = 'OHLC_%s'%i,
        increasing=dict(line=dict(color=INCREASING_COLOR[i])),
        decreasing=dict(line=dict(color=DECREASING_COLOR[i])),
        )
        data.append(idict)
        colors.append(list(df_list[i]['state'].map({True:INCREASING_COLOR[i],False:DECREASING_COLOR[i]})))
        
    layout = go.Layout(
        xaxis = dict(rangeslider = dict(visible = False)),
        title = df_list[0].name,
        titlefont = dict(size = 30),
        yaxis = dict( domain = [0, 0.2], showticklabels = True),
        yaxis2 = dict( domain = [0.2, 0.97] ),
        legend = dict( orientation = 'h', y=0.97, x=0, yanchor='bottom'),
        margin = dict( t=60, b=40, r=40, l=40 ),
        plot_bgcolor = 'rgb(250, 250, 250)'
    )
    fig = dict(data=data, layout=layout)
    
    for i in range(len(df_list)):
        fig['data'].append(dict( x=pd.to_datetime(df_list[i]['trade_date']), y=df_list[i].vol, marker=dict( color=colors[i] ),
                            type='bar', yaxis='y', name='vol_%s'%i ))

    name = '{}.html'.format(df_list[0].name.split('-')[1])
    py.plot( fig, filename = name)
    return df_list

# 【实测】

In [13]:
get_allNdays()[0].head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
ts_code,,,,,,,,,,,,
300119.SZ,300119.SZ,20190328,13.97,14.13,13.00,13.25,13.97,-0.72,-5.1539,55164.38,75041.645,False
601818.SH,601818.SH,20190328,4.01,4.02,3.97,4.00,4.02,-0.02,-0.4975,626640.92,250046.032,False
300125.SZ,300125.SZ,20190328,17.85,18.58,16.56,16.88,17.98,-1.10,-6.1179,343317.09,610368.035,False
002486.SZ,002486.SZ,20190328,4.19,4.24,4.12,4.12,4.17,-0.05,-1.1990,103609.00,43382.547,False
002482.SZ,002482.SZ,20190328,6.08,6.22,6.06,6.09,6.19,-0.10,-1.6155,113440.51,69629.194,True


In [14]:
get_allNdays(date='20190325', N=2, print_mode=True)[0].head()

【20190325】	【20190322】	

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
ts_code,,,,,,,,,,,,
600000.SH,600000.SH,20190322,11.49,11.51,11.31,11.39,11.46,-0.07,-0.6108,354364.46,403972.783,False
600004.SH,600004.SH,20190322,14.40,15.00,14.31,14.89,14.43,0.46,3.1878,220079.97,324663.261,True
600006.SH,600006.SH,20190322,4.52,4.98,4.50,4.98,4.53,0.45,9.9338,606897.36,294476.945,True
600007.SH,600007.SH,20190322,14.68,14.95,14.63,14.81,14.76,0.05,0.3388,32909.36,48766.239,True
600008.SH,600008.SH,20190322,4.21,4.35,4.14,4.28,4.24,0.04,0.9434,780633.15,332835.785,True


# 筛选倍量柱

In [15]:
selected = get_blz()
print(list(selected.keys())[0])
list(selected.values())[0][:5]

20190329


['603338.SH', '300647.SZ', '603500.SH', '002892.SZ', '002905.SZ']

In [58]:
selected = get_blz(date='20190322', v_pct_min=0.99, v_pct_max=1.01, c_pct_min=1.05, ignore=[], print_mode=True)

600191.SH	v:1.01	c:1.05
600356.SH	v:1.0	c:1.07
600422.SH	v:0.99	c:1.1


In [61]:
df = plotly_ohlc(list(selected.values())[0][0])
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
0,600191.SH,20180604,6.01,6.13,5.93,5.96,5.93,0.03,0.51,40004.93,24104.799,False
1,600191.SH,20180605,5.93,6.01,5.89,5.98,5.96,0.02,0.34,28872.31,17185.965,True
2,600191.SH,20180606,5.98,6.02,5.86,5.89,5.98,-0.09,-1.51,33153.44,19595.444,False
3,600191.SH,20180607,5.88,5.92,5.79,5.80,5.89,-0.09,-1.53,31678.63,18535.841,False
4,600191.SH,20180608,5.80,5.81,5.52,5.60,5.80,-0.20,-3.45,48944.44,27572.412,False


In [18]:
df = plotly_ohlc(code='600191.SH', date='20190322', period=100)
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
0,600191.SH,20181212,4.53,4.61,4.49,4.50,4.58,-0.08,-1.7467,36063.81,16376.575,False
1,600191.SH,20181213,4.50,4.57,4.45,4.49,4.50,-0.01,-0.2222,29469.86,13324.881,False
2,600191.SH,20181214,4.51,4.52,4.35,4.35,4.49,-0.14,-3.1180,32646.85,14455.650,False
3,600191.SH,20181217,4.32,4.40,4.25,4.34,4.35,-0.01,-0.2299,22400.87,9661.699,True
4,600191.SH,20181218,4.27,4.40,4.26,4.35,4.34,0.01,0.2304,18675.40,8108.025,True


# 筛选黄金柱

In [25]:
selected = get_hjz()
print(list(selected.keys())[0])
list(selected.values())[0][:5]

20190329


['000651.SZ', '000987.SZ', '001979.SZ', '002081.SZ', '002203.SZ']

In [20]:
# selected = get_blz(date='20190322', ignore=[], print_mode=True)

# 筛选缩量群

In [26]:
selected = get_slq()

In [32]:
selected = get_slq(date='20190325', period=5, rough=0.8, ignore=[], print_mode=True)

000590.SZ
002438.SZ
600150.SH


# 筛选平量群

In [49]:
selected = get_plq()

In [52]:
selected = get_plq(date='20190325', period=5, rough=0.8, ignore=[], print_mode=True)

# 筛选梯量群

In [54]:
selected = get_tlq()

In [55]:
selected = get_tlq(date='20190325', period=5, rough=0.8, ignore=[], print_mode=True)

# N天中满足条件的次数

In [34]:
count = countN()
i = 0
for code, times in count.items():
    i += 1
    if i<10:print(code,times)

000002.SZ 1
000020.SZ 4
000023.SZ 3
000031.SZ 4
000037.SZ 1
000045.SZ 4
000059.SZ 3
000061.SZ 3
000070.SZ 1


In [35]:
count = countN(func=get_blz, N=2, parameters={'date':'20190318','v_pct_min':0.95, 'v_pct_max':1.05, 'c_pct_min':1.05})
i = 0
for code, times in count.items():
    i += 1
    if i<10:print(code,times)

000006.SZ 1
000401.SZ 1
000596.SZ 2
000671.SZ 1
000673.SZ 1
002011.SZ 1
002054.SZ 1
002367.SZ 1
002596.SZ 1


In [36]:
count = countN(func=get_hjz, N=2, parameters={'date':'20190318'})
i = 0
for code, times in count.items():
    i += 1
    if i<10:print(code,times)

000001.SZ 1
000016.SZ 1
000027.SZ 1
000062.SZ 1
000428.SZ 1
000539.SZ 1
000551.SZ 1
000589.SZ 1
000598.SZ 1


In [37]:
count = countN(func=get_slq, N=2, parameters={'date':'20190318','period':5, 'rough':0.9})
i = 0
for code, times in count.items():
    i += 1
    if i<10:print(code,times)

600036.SH 1


# N天K线图

In [38]:
df = Nplotly_ohlc(code='600191.SH')
df[0].head()

,trade_date,open,high,low,close,vol,amount,state
1,20180604,6.01,6.13,5.89,5.98,68877.2,41290.8,False
3,20180606,5.98,6.02,5.79,5.8,64832.1,38131.3,False
5,20180608,5.8,5.81,5.5,5.55,74681.4,41898.2,False
7,20180612,5.54,5.62,5.35,5.38,62470.1,34389.2,False
9,20180614,5.4,5.47,5.07,5.16,66181,34969.7,False


In [39]:
df[1].head()

,trade_date,open,high,low,close,vol,amount,state
2,20180605,5.93,6.02,5.86,5.89,62025.8,36781.4,False
4,20180607,5.88,5.92,5.52,5.6,80623.1,46108.3,False
6,20180611,5.59,5.62,5.5,5.61,50048.1,27900,True
8,20180613,5.6,5.6,5.35,5.4,63064.9,34289.3,False
10,20180615,5.38,5.43,4.64,4.64,101137,50223.1,False


# 耗时
```python
%%time 将会给出cell的代码运行一次所花费的时间

%time 将会给出当前行的代码运行一次所花费的时间

%timeit 使用Python的timeit模块，它将会执行一个语句100，000次(默认情况下)，然后给出运行最快3次的平均值。
```

In [41]:
%time get_date();

CPU times: user 21 µs, sys: 1 µs, total: 22 µs
Wall time: 25.7 µs


'20190329'